# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [5]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [6]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [7]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [29]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages, or Contact us page, or news page or product and services page.\n"
link_system_prompt += "You should respond in JSON as in the examples below:"
link_system_prompt += """
    Template Example
    
    Links
    {
        "links": [
            {"type": "about page", "url": "https://full.url/goes/here/about"},
            {"type": "careers page": "url": "https://another.full.url/careers"}
        ]
    }

    Example 2 (Dell)

    Links
    {
        "links": [
            {"type": "about page", "url": "https://www.dell.com/en-au/lp/dt/who-we-are"},
            {"type": "careers page": "url": "https://jobs.dell.com/en"},
            {"type": "contact us page": "url": "https://www.dell.com/en-au/lp/contact-us"},
            {"type": "services page": "url": "https://www.dell.com/en-au/lp/dt/services"},
            {"type": "news page": "url": "https://www.dell.com/en-us/dt/corporate/newsroom.htm#country=en-us"}
        ]
    }
    
    
    """

In [30]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages, or Contact us page, or news page or product and services page.
You should respond in JSON as in the examples below:
    Template Example

    Links
    {
        "links": [
            {"type": "about page", "url": "https://full.url/goes/here/about"},
            {"type": "careers page": "url": "https://another.full.url/careers"}
        ]
    }

    Example 2 (Dell)

    Links
    {
        "links": [
            {"type": "about page", "url": "https://www.dell.com/en-au/lp/dt/who-we-are"},
            {"type": "careers page": "url": "https://jobs.dell.com/en"},
            {"type": "contact us page": "url": "https://www.dell.com/en-au/lp/contact-us"},
            {"type": "services page": "url": "https://www.dell.com/en-au/lp/dt/services"},
       

In [31]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [32]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [33]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [34]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/sesame/csm-1b',
 '/mistralai/Mistral-Small-3.1-24B-Instruct-2503',
 '/ds4sd/SmolDocling-256M-preview',
 '/manycore-research/SpatialLM-Llama-1B',
 '/google/gemma-3-27b-it',
 '/models',
 '/spaces/sesame/csm-1b',
 '/spaces/stabilityai/stable-virtual-camera',
 '/spaces/prs-eth/thera',
 '/spaces/jasperai/LBM_relighting',
 '/spaces/ameerazam08/Gemini-Image-Edit',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/nvidia/PhysicalAI-Robotics-GR00T-X-Embodiment-Sim',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/open-r1/codeforces-cots',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/am

In [35]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'news page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'contact us page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'contact us page', 'url': 'https://huggingface.co/chat'},
  {'type': 'product and services page',
   'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [36]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [37]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'posts page', 'url': 'https://huggingface.co/posts'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact us page', 'url': 'https://huggingface.co/contact'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'discussion forum page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'ur

In [38]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [49]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short, snarky, ironic brochure about the company for prospective customers, investors, and recruits. \
Your tone should be sarcastic, dry, and witty—poking fun at corporate jargon, meaningless buzzwords, and the general absurdity of corporate culture. \
Highlight the company’s quirks, exaggerated promises, and work-life balance 'initiatives' with a healthy dose of irony. \
Respond in markdown. Include sarcastic details about company culture, customers, news, products/services, contact details and careers/jobs if available."

In [82]:
def get_brochure_user_prompt(company_name, url, language):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown in the {language} language.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [81]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

TypeError: get_brochure_user_prompt() missing 1 required positional argument: 'language'

In [83]:
def create_brochure(company_name, url, language):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url, language)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [84]:
create_brochure("HuggingFace", "https://huggingface.co","Spanish")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'contact us page', 'url': 'https://huggingface.co/contact'}, {'type': 'news page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


```markdown
# Bienvenido a Hugging Face: Donde la IA se convierte en una realidad... o eso dicen.

¡Hola, futuro cliente, inversionista o empleado! ¿Listo para unirse a la "comunidad de IA que construye el futuro"? O al menos, eso es lo que pregonan mientras incrementan los modelos, conjuntos de datos y - ¡oh sí! - también aplicaciones. Pero, ¿quién necesita un futuro cuando tienes abrazos virtuales y jerga corporativa para mantenerte caliente?

---

## Lo Que Hacemos

- **Modelos**: Porque uno nunca es suficiente. Explora más de **1 millón** de modelos y maravíllate con la variedad. ¿Te has preguntado alguna vez cuántos de ellos realmente funcionan? 🤔
  
- **Conjuntos de Datos**: Aquí tenemos **250,000+ conjuntos de datos** que son tan fascinantes como leer la lista de ingredientes en un yogurt sin grasa. 

- **Espacios**: Por supuesto, también tenemos "espacios" donde puedes "colaborar". Lo que realmente significa que puedes asistir a numerosas reuniones en línea cuando podrías estar desarrollando algo útil... como una mezcla de café.

---

## Cultura Corporativa

¡Bienvenido a la **cultura laboral del futuro**! Nuestros colaboradores son tan apasionados por la IA que ni siquiera notan cuando trabajan tarde porque están esperando que su código se compile. Y no te preocupes, tenemos **iniciativas de equilibrio entre trabajo y vida personal**: como llevar tu laptop a la cama, porque, ¿quién no ama un poco de insomnio?

Con más de **50,000 organizaciones** usando nuestro servicio (incluyendo algunos nombres que has escuchado en anuncios de televisión), puedes estar seguro de que estamos haciendo algo... pero no te preguntes qué exactamente.

---

## Servicios Espectaculares

- **Compute**: ¡Despliega tu IA en menos tiempo del que toma un espresso! Desde **$0.60/hora**. Ahora, eso es un precio justo por un pequeño trozo del futuro, ¿verdad?
  
- **Enterprise**: ¿Eres una gran empresa? ¡Perfecto! Comienza con solo **$20/usuario/mes** y recibe soporte prioritario. Porque, a lo largo de los años, aprendimos que la rapidez es todo, excepto en la resolución de tickets.

---

## ¿Listo Para Unirte?

Si soportas un poco de caos y confusión, visita nuestra página de **empleos**. Pero prepárate, la única respuesta que obtendrás podría ser un generador de respuestas automáticas... ¡genial! 

---

### Contáctanos

¡Oh, espera! Si tienes alguna pregunta, puedes encontrarnos en nuestras redes sociales o en nuestra encantadora **página 404** porque, a veces, los abrazos son lo único que queda. 😬

> **Hugging Face** — Construyendo el futuro, un modelo a la vez... o al menos intentándolo.

--- 

¡Así que únete a nosotros! Porque al final del día, ¡solo queremos que te sientas bien mientras abrazamos a la IA!
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [45]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    # for chunk in stream:
    #     print (chunk.choices[0].delta.content or '', end='')
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [46]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'posts page', 'url': 'https://huggingface.co/posts'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact us page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': '

# Welcome to Hugging Face: AI’s Not-So-Huggy Playground

## The AI Community Building the Future… or Just Another Buzzword?

At Hugging Face, we’re revolutionizing how you collaborate, innovate, and possibly even define your existence in this wild AI playground. Forget everything you thought you knew about machines—they’re our new **friends!** (And they don’t even need snacks.) 

### What We Offer (Aside from Struggling to Keep It Real)

- **1M+ Models**: Because who doesn’t need a plethora of models that are all instantly upgradable? Our models change more often than you can blink—don’t worry, great productivity products come with a generous side of confusion!

- **Datasets and Spaces**: We have so many datasets, they actually **outnumber** your hopes and dreams. Use them wisely, or maybe just throw them at the wall and see what sticks!

- **AI Apps Galore**: From speech generation to image enhancement, we’ve got everything except for an app that helps you figure out why you're striving to build a future with AI (your guess is as good as ours!).

### Why Choose Us? 

Because we **care**! Maybe more than we should. Over **50,000 organizations** are already *allegedly* using us, from tech giants to the small non-profit you barely remember donating to in 2019. If that doesn’t convince you, I don’t know what will!

### Our Amazing Work-Life Balance Initiatives

We believe in finding the delicate balance between *working hard and hardly working.* Join us for a vibrant mix of “let’s supercharge your productivity!” meetings and “oops, I accidentally took a two-hour lunch break” kind of days. Forget regular 9-to-5; it’s all about *flexibility*—which means you can stress about getting everything done any time of the day or night!

### Employee Perks

- **Priority Support**: Because if you’re going to get lost in the corporate maze, at least you deserve to have a flashlight, right?
  
- **Dedicated Support**: Need help? You’ll find our support team dedicated enough to point you in the right direction—just don’t expect them to do your work for you!

- **Single Sign-On**: Because typing in all those passwords just wastes precious *working* moments when you could be scrolling through someone’s pet meme collection instead.

### Join the Hugging Face Family!

Let’s embark on this *innovative* journey together! Whether you’re looking for a last-minute career switch, a way to break into the world of AI while still clinging to your sanity, or just an excuse to use the phrase “machine learning” in casual conversation, we’ve got you covered.

*Disclaimer: We cannot guarantee your happiness or success here, but we can promise a hearty dose of sarcasm and limitless opportunities to redefine your career path—or choose a completely contradictory one. Happy creating!* 

--- 

### Ready to Join Us? 

Sign up on our website to dive right into the futuristic chaos of AI that promises more than it probably should!

# Spanish

In [73]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(result)

In [74]:
broch = create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'contact us page', 'url': 'https://huggingface.co/chat'}, {'type': 'discuss page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'# Welcome to HuggingFace, Where AI Dreams Become Overhyped Reality!\n\n## The AI Community ... or is it just a Club?\nIn a stunningly original venture, **Hugging Face** sets itself up as the *"AI community building the future."* Yes, that\'s right — we\'ve taken corporate jargon and applied it to algorithms. Who needs tangible social interactions when you can collaboratively train machine learning models from the comfort of your solitary confinement?\n\n## Models, Datasets, and Other Corporate Buzzwords\nWith more than **1 million models** at your disposal, feel free to browse through a dizzying amount of half-baked ideas masquerading as cutting-edge tech. It\'s like a buffet of potential. Will you pick the *“mistralai/Mistral-Small-3.1-24B-Instruct-2503”* (that\'s a mouthful) or the trending *“google/gemma-3-27b-it”*? The choice is yours, and no, you won\'t find anyone who truly knows what these names mean.\n\n### The Magical World of Spaces\nOpen the door to the **Spaces** section a

In [75]:
def translate_to_spanish(text):
    translation_prompt = f"Translate the following text to Spanish:\n\n{text}"
    response = openai.chat.completions.create(
        model=MODEL, 
        messages=[
            {"role": "system", "content": "You are a helpful assistant proficient in translating English to Spanish."},
            {"role": "user", "content": translation_prompt}
        ],
    )
    translated_text = response.choices[0].message.content
    return translated_text

In [76]:
translated_brochure = translate_to_spanish(broch)
display(Markdown(translated_brochure))

Ninguno

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>